In [1]:
import scanpy as sc
import muon as mu
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import roc_curve, auc

/home/wuxinchao/software/miniconda3/envs/pytorch-gpu/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# using different methods to get the initial graph construction of the cells

def permutate_graph_construct(adata, dim=50, init_neighbor_method='louvain', res=1, batch_size=256, device='cuda'):
    """To use contrastive learning, we need to permutate the graph construct of the data."""
    adata_perm = adata.copy()
    # Permutate the cell expression matrix and get the new graph cnostruct of the data
    adata_perm.X = np.random.permutation(adata_perm.X)
    if init_neighbor_method == 'louvain':
        # Get the louvain graph construct of the data
        expression = adata_perm.X
        sc.pp.neighbors(adata_perm, use_rep='X', n_neighbors=30)
        sc.tl.louvain(adata_perm, resolution=res)


def get_contrastive_loss(adata, adata_perm, dim=50, batch_size=256, device='cuda'):
    """Get the contrastive loss of the data."""
    # Get the contrastive loss

In [ ]:
from utils import define_positive_pairs

def constrastive_loss(pred_embedding, pos_pairs, neg_pairs, device='cuda'):
    """
    Define the loss function for contrastive learning.
    The optimization goal is to make the distance between positive
    pairs smaller and the distance between negative pairs larger.
    """
    # Get the positive pairs and negative pairs
    pos_pairs, neg_pairs = define_positive_pairs(pred_embedding, device=device)
    # Get the distance of the positive pairs and negative pairs
    pos_dist = torch.sum((pred_embedding[pos_pairs[:, 0]] - pred_embedding[pos_pairs[:, 1]]) ** 2, dim=1)
    neg_dist = torch.sum((pred_embedding[neg_pairs[:, 0]] - pred_embedding[neg_pairs[:, 1]]) ** 2, dim=1)
    # Get the loss
    loss = torch.mean(pos_dist) - torch.mean(neg_dist)
    return loss